# ECS Web Scraping and Excel Formatting Script

This script, named **"ECS Statewide Longitudinal Data Systems Scraper"**, performs the following tasks:

1. **Web Scraping**: The script scrapes data from a specific webpage (`https://reports.ecs.org/comparisons/statewide-longitudinal-data-systems-2024`) that contains a table with information about Statewide Longitudinal Data Systems.
2. **Data Extraction**: It extracts the data from the table, including any hyperlinks.
3. **Data Cleaning**: The script cleans the data by removing unwanted text from the headers.
4. **Excel Export**: The cleaned data is exported to an Excel file with two sheets:
   - **URL**: Contains the extracted URLs in plain text.
   - **original**: Contains the original text with hyperlinks formatted in Excel.
5. **Formatting**: In the Excel file, the header rows and row 53 in each sheet are bolded. In the "original" sheet, hyperlinks are styled with the classic blue, underlined text.

The script uses the following libraries:
- `requests` for fetching the webpage content.
- `BeautifulSoup` from `bs4` for parsing HTML.
- `pandas` for data manipulation.
- `openpyxl` for creating and formatting the Excel file.
- `os` for determining the path to the user's desktop.

Run the script in a Python environment to generate the Excel file on your desktop with the specified formatting.

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
import os

# URL of the page to scrape
url = 'https://reports.ecs.org/comparisons/statewide-longitudinal-data-systems-2024'

# Send a GET request to fetch the raw HTML content
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table element by its ID
table = soup.find('table', id='mainTable')

# Extract headers and clean them
headers = [th.get_text(strip=True).replace("arrow_upward", "") for th in table.find_all('th')]

# Extract rows for the URL DataFrame
rows = []
original_rows = []
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all('td')
    row = []
    original_row = []
    for cell in cells:
        # Extract the text from each cell
        text = cell.get_text(strip=True)
        # Check if the cell contains a link
        link = cell.find('a')
        if link:
            row.append(link['href'])  # Add URL to the row
            original_row.append((link['href'], link.get_text(strip=True)))  # Add tuple with URL and text to the original_row
        else:
            row.append(text)
            original_row.append((None, text))  # Add tuple with None and text for non-links
    rows.append(row)
    original_rows.append(original_row)

# Create DataFrames
df = pd.DataFrame(rows, columns=headers)

# Create an Excel workbook and add sheets
wb = Workbook()
ws_url = wb.active
ws_url.title = "URL"
ws_original = wb.create_sheet(title="original")

# Bold font style
bold_font = Font(bold=True)

# Add data to the URL sheet with headers bolded
for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = ws_url.cell(row=r_idx, column=c_idx, value=value)
        if r_idx == 1 or r_idx == 53:  # Bold headers and row 53
            cell.font = bold_font

# Add headers to the original sheet and bold them
for c_idx, header in enumerate(headers, 1):
    cell = ws_original.cell(row=1, column=c_idx, value=header)
    cell.font = bold_font

# Add data to the original sheet with hyperlinks and bold row 53
for r_idx, row in enumerate(original_rows, 2):  # Start from row 2 to account for headers
    for c_idx, (url, text) in enumerate(row, 1):
        cell = ws_original.cell(row=r_idx, column=c_idx, value=text)
        if url:
            cell.hyperlink = url
            cell.font = Font(color="0000FF", underline="single")  # Set hyperlink style
        if r_idx == 53:  # Bold row 53
            cell.font = bold_font

# Save the workbook to a file on the desktop
desktop_path = os.path.join(os.path.expanduser("~"), 'Desktop')
output_path = os.path.join(desktop_path, 'ECS_SLDS_scraped_table.xlsx')
wb.save(output_path)

print(f"Data exported successfully to {output_path}")

Data exported successfully to C:\Users\sbaser\Desktop\ECS_SLDS_scraped_table.xlsx
